# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output,prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/waffle_schema.sql")
if not sql_schema:
    print("Warning: Could not load waffle_schema.sql. Lab may not function correctly.")

2025-10-02 14:20:24,025 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [3]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""
As a senior Python developer, using {sql_schema} as the database schema generate the followings:\
- Pydantic models for API data validation : provide request and response bodies\
- FastAPI endpoints: Create a Full CRUD (Create, Read, Update, Delete) endpoints for the users table\
- In-memory Database: A simple python list to act as a temporary database\

Output:
- a single python script

Context:
Schema: {sql_schema}
"""
in_memory_api_prompt = prompt_enhancer(in_memory_api_prompt)
print(in_memory_api_prompt)
print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/waffle_main_in_memory.py",overwrite=True)
else:
    print("Skipping API generation because schema is missing.")

2025-10-02 14:20:36,074 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<prompt>

  <persona>
    You are a Senior Python Backend Engineer with deep expertise in FastAPI, Pydantic, and rapid prototyping. Your goal is to deliver clean, production-ready code that strictly follows best practices.
  </persona>

  <context>
    You are provided the following SQL schema:

    -- User Roles Table
    CREATE TABLE user_roles (
        role_id INTEGER PRIMARY KEY,
        role_name TEXT NOT NULL UNIQUE
    );

    -- Users Table
    CREATE TABLE users (
        user_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name  TEXT NOT NULL,
        email      TEXT NOT NULL UNIQUE,
        hire_date  TEXT NOT NULL,
        role_id    INTEGER NOT NULL,
        bio        TEXT,
        FOREIGN KEY (role_id) REFERENCES user_roles(role_id),
        CHECK (email LIKE '%@%')
    );

    -- Tasks Per Role Table
    CREATE TABLE tasks_per_role (
        task_id             INTEGER PRIMARY KEY AUTOINCREMENT,
        role_id             INTEGER NOT NULL,
     

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [5]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""
# As an expert Python developer, your task is to generate 2 separate pieces of code:\

Task:

1. Create python classes that maps to the database tables {sql_schema}
2. the boilerplate code to create a database engine, session maker, and a dependency function like get_db() for use in FastAPI. Include db.append() methods as well

Conditions:
The output should be 2 distinct, will commented python code snippets
"""

db_code_prompt = prompt_enhancer(db_code_prompt)
print(db_code_prompt)

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
    save_artifact(generated_db_code, "app/waffle_helper.py")
else:
    print("Skipping DB code generation because schema is missing.")

2025-10-02 12:57:47,653 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<persona>
You are an expert Python developer who specializes in SQLAlchemy ORM design and FastAPI back-end architecture.
</persona>

<context>
Database schema to model:

```sql
CREATE TABLE user_roles (
    role_id   INTEGER PRIMARY KEY,
    role_name TEXT NOT NULL UNIQUE
);

CREATE TABLE users (
    user_id    INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name  TEXT NOT NULL,
    email      TEXT NOT NULL UNIQUE,
    hire_date  TEXT NOT NULL,
    role_id    INTEGER NOT NULL,
    bio        TEXT,
    FOREIGN KEY (role_id) REFERENCES user_roles(role_id),
    CHECK (email LIKE '%@%')
);

CREATE INDEX idx_users_role_id ON users(role_id);
```

Key requirements and constraints:
• Produce two DISTINCT Python code snippets.  
• Each snippet must be thoroughly commented.  
• Snippet 1: SQLAlchemy ORM classes mapping precisely to the two tables above (include indexes, constraints, relationships, etc.).  
• Snippet 2: Boilerplate that:
  – Creates an async or sync SQLAlchemy engine 

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

In [8]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
main_py = load_artifact("app/waffle_main.py")
pytest_prompt = f"""
You are tasked with creating a comprehensive pytest test suite for a FastAPI application that manages users, roles, and tasks. The application is called "Waffle Tech Suite API" and includes full CRUD operations for three main resources.

## Application Overview

The FastAPI application {main_py} has the following structure:

### Database Models:
1. **UserRole** - Stores role information (role_id, role_name)
2. **User** - Stores user information (user_id, first_name, last_name, email, hire_date, role_id, bio)
3. **TaskPerRole** - Stores tasks assigned to roles (task_id, role_id, task_description, completion_timeline)

### API Endpoints (16 total):

**Roles (3 endpoints):**
- POST `/roles/` - Create a new role
- GET `/roles/` - List all roles
- GET `/roles/[role_id] - Get role by ID

**Users (6 endpoints):**
- POST `/users/` - Create a new user
- GET `/users/` - List all users
- GET `/users/[user_id] - Get user by ID
- PUT `/users/[user_id]` - Full update of user
- PATCH `/users/[user_id]` - Partial update of user
- DELETE `/users/[user_id] - Delete user

**Tasks (7 endpoints):**
- POST `/tasks/` - Create a new task
- GET `/tasks/` - List all tasks
- GET `/tasks/[task_id]` - Get task by ID
- GET `/tasks/role/[role_id]` - Get all tasks for a specific role
- PUT `/tasks/[task_id]` - Full update of task
- PATCH `/tasks/[task_id]` - Partial update of task
- DELETE `/tasks/[task_id]` - Delete task

### Validation Rules:
- **Email**: Must be valid email format, minimum 6 characters, unique
- **Names**: Minimum 2 characters
- **Dates** (hire_date, completion_timeline): Must match YYYY-MM-DD format
- **Role names**: Minimum 2 characters, unique
- **Task descriptions**: Minimum 5 characters, maximum 500 characters
- **Bio**: Optional, maximum 500 characters
- **Foreign keys**: role_id must exist in user_roles table

### Expected Error Responses:
- **400**: Duplicate email, duplicate role name, invalid role_id
- **404**: Resource not found (user, role, or task)
- **422**: Validation errors (invalid date format, field length violations)

## Task Requirements

Generate a complete pytest test suite (`test_waffle_main.py`) that includes:

### 1. Test Setup
- Use `pytest.fixture` for test database setup with SQLite in-memory database
- Use `TestClient` from `fastapi.testclient` for API testing
- Create fixtures for:
  - Database session override
  - Sample roles for testing
  - Sample users for testing
  - Sample tasks for testing

### 2. Test Coverage for Each Endpoint

For **EACH** of the 16 endpoints, create tests for:

#### Happy Path Tests:
- Successful creation/retrieval/update/deletion
- Verify correct status codes (200, 201 if applicable)
- Verify response data structure matches expected schema
- Verify data is correctly stored in database

#### Edge Case Tests:
- **Validation errors**: Invalid formats, missing required fields, field length violations
- **Duplicate entries**: Attempting to create duplicates where uniqueness is required
- **Foreign key violations**: Invalid role_id references
- **Not found scenarios**: Attempting to access non-existent resources
- **Boundary conditions**: Maximum length strings, edge case dates
- **Partial updates**: PATCH with various field combinations

### 3. Specific Test Scenarios to Include

**Role Tests:**
- Create role with valid data
- Create role with duplicate name (should fail)
- Create role with name too short (should fail)
- Get all roles returns list
- Get specific role by ID
- Get non-existent role (404)

**User Tests:**
- Create user with all fields including bio
- Create user without optional bio field
- Create user with duplicate email (should fail)
- Create user with invalid email format (should fail)
- Create user with invalid date format (should fail)
- Create user with non-existent role_id (should fail)
- Get all users returns list with correct structure
- Get specific user by ID
- Full update (PUT) changes all fields
- Partial update (PATCH) with single field
- Partial update (PATCH) with multiple fields
- Update user email to existing email (should fail)
- Delete user successfully
- Delete non-existent user (404)

**Task Tests:**
- Create task with valid data
- Create task with invalid role_id (should fail)
- Create task with description too short (should fail)
- Create task with invalid date format (should fail)
- Get all tasks
- Get task by ID
- Get all tasks for a specific role (should return filtered list)
- Get tasks for non-existent role (404)
- Full update (PUT) task
- Partial update (PATCH) task description only
- Delete task successfully
- Delete non-existent task (404)

### 4. Test Organization
- Group tests by resource using classes (TestRoles, TestUsers, TestTasks)
- Use descriptive test names following pattern: `test_<action>_<resource>_<scenario>`
- Add docstrings to complex tests explaining what is being tested

### 5. Additional Requirements
- Use `assert` statements with clear failure messages
- Test both status codes AND response body content
- Clean up test data appropriately between tests
- Use parametrize where appropriate for testing multiple similar scenarios
- Include integration tests that test relationships (e.g., creating user with role, getting tasks by role)

## Output Format

Provide the complete `test_waffle_main.py` file with:
1. All necessary imports
2. Fixture definitions
3. Test classes organized by resource
4. All test methods with clear names and docstrings
5. Comments explaining complex test logic

The test suite should be production-ready and executable with: `pytest test_waffle_main.py -v`
Generate comprehensive pytest tests that achieve high code coverage and thoroughly validate all API functionality, business logic, and error handling.
Remove any special characters or formatting issues from the output.
"""

pytest_prompt = prompt_enhancer(pytest_prompt)
pytest_code = get_completion(pytest_code, client, model_name, api_provider)
print(pytest_code)
save_artifact(pytest_code, "tests/test_waffle_main.py",overwrite=True)

2025-10-02 14:34:25,599 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


```python
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from httpx import AsyncClient
from app.main import app
from app.models import Base, Role, User, Task
from app.dependencies import get_db

# Configure test database
DATABASE_URL = "sqlite:///:memory:"
engine = create_engine(DATABASE_URL)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create all tables for the test database
Base.metadata.create_all(bind=engine)

@pytest.fixture(scope="function")
def db() -> Session:
    """Provides a new SessionLocal for each test, ensuring isolation."""
    Base.metadata.create_all(bind=engine)
    db = TestingSessionLocal()
    try:
        yield db
    finally:
        db.rollback()
        db.close()

@pytest.fixture(scope="function")
def client(db: Session):
    """Provides a TestC

WindowsPath('C:/Users/labadmin/Desktop/Repository/AG-AISOFTDEV/artifacts/tests/test_waffle_main.py')

In [9]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.

security_vulnerability_report_prompt = f"""
You are an experienced application security expert specializing in web API security, OWASP Top 10 vulnerabilities, and secure coding practices. Your task is to conduct a comprehensive security review of a FastAPI application that manages users, roles, and tasks.

Context: {main_py}

Conduct a thorough security assessment covering the following areas:
1. Injection Vulnerabilities

SQL Injection: Analyze all database queries for potential SQL injection risks
NoSQL Injection: Check for any NoSQL query vulnerabilities (if applicable)
Command Injection: Look for any OS command execution risks
LDAP/XML Injection: Identify other injection vectors

2. Authentication and Authorization

Authentication Mechanism: Evaluate the presence and strength of authentication
Authorization Controls: Check if users can access resources they shouldn't
Session Management: Assess session handling and token security
Password Security: Review password storage and handling practices

3. Input Validation and Sanitization

Validation Coverage: Identify missing or weak input validation
Data Type Validation: Check if all inputs are properly typed and validated
Length Restrictions: Verify appropriate field length limits
Format Validation: Assess date, email, and other format validations
Boundary Conditions: Test edge cases and boundary values

4. Data Exposure and Privacy

Sensitive Data Handling: Check for exposed sensitive information
Error Messages: Review if error messages leak internal information
Logging: Assess if logs contain sensitive data
API Response Data: Check for over-exposure of data in responses

5. Access Control Issues

Broken Object Level Authorization (BOLA): Check if users can access other users' data
Broken Function Level Authorization (BFLA): Verify endpoint access controls
Insecure Direct Object References (IDOR): Look for predictable ID-based access

6. Security Misconfiguration

CORS Configuration: Review Cross-Origin Resource Sharing settings
Error Handling: Check if errors expose stack traces or sensitive info
Default Configurations: Identify any insecure default settings
Security Headers: Assess missing security headers

7. Database Security

Connection Security: Review database connection string handling
Parameterized Queries: Verify use of parameterized queries vs string concatenation
Database Credentials: Check how credentials are stored and accessed
Data Encryption: Assess if sensitive data is encrypted at rest

8. Business Logic Vulnerabilities

Race Conditions: Identify potential race condition issues
Mass Assignment: Check for uncontrolled object property assignment
State Management: Review transaction handling and state consistency
Data Integrity: Assess foreign key constraints and referential integrity

9. API-Specific Vulnerabilities

Rate Limiting: Check for absence of rate limiting
API Versioning: Review API version management
HTTP Methods: Verify appropriate HTTP method usage and restrictions
Content Type Validation: Check Content-Type header validation

10. Dependencies and Configuration

Dependency Vulnerabilities: Identify potentially vulnerable libraries
Configuration Management: Review how configuration is handled
Environment Variables: Check for hardcoded secrets or credentials

Output Format
Provide your findings in a structured markdown document with the following format: 
1. Executive Summary
2. Critical Vulnerabilities
3. High Severity Issues
4. Medium Severity Issues
5. Low Severity Issues
6. Security Best Practices Recommendations

"""


security_vulnerability_report = get_completion(security_vulnerability_report_prompt, client, model_name, api_provider)
print(security_vulnerability_report)
save_artifact(security_vulnerability_report, "artifacts/security_review.md",overwrite=True)

# Security Assessment Report: Waffle Tech Suite API

## 1. Executive Summary

This security assessment of the Waffle Tech Suite API, a FastAPI application, evaluates its resilience against common web application vulnerabilities identified by OWASP. The application provides CRUD operations for managing users, roles, and tasks. The assessment covers areas such as injection vulnerabilities, authentication and authorization, input validation, data exposure, access control, security misconfigurations, database security, business logic vulnerabilities, API-specific vulnerabilities, and dependency management.

## 2. Critical Vulnerabilities

### 2.1 Injection Vulnerabilities
- **SQL Injection**: The application uses SQLAlchemy ORM, which inherently protects against SQL injection through parameterized queries. However, ensure that no raw SQL queries are introduced without proper parameterization.

### 2.2 Authentication and Authorization
- **Authentication**: There is no authentication mechani

WindowsPath('C:/Users/labadmin/Desktop/Repository/AG-AISOFTDEV/artifacts/security_review.md')

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.